In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
import glob
import random
from collections import defaultdict
from pathlib import Path

from IPython.display import display

import pandas as pd
import dask

from tqdm import tqdm
import jax
jax.config.update('jax_platform_name', 'cpu')
# jax.config.update('jax_log_compiles', True)
# jax.config.update("jax_debug_nans", True)
# jax.config.update("jax_enable_x64", True)

In [2]:


sys.path.append("../..")

from lib import utils as U
from lib.ehr.dataset import load_dataset, load_dataset_scheme, Dataset
from lib.ehr.interface import Patients
from lib.ehr.concepts import DemographicVectorConfig, LeadingObservableConfig


In [3]:
import logging
logging.root.level = logging.INFO

In [4]:
# Assign the folder of the dataset to `DATA_FILE`.

HOME = os.environ.get('HOME')
DATA_DIR = f'{HOME}/GP/ehr-data'
SOURCE_DIR = os.path.abspath("..")
sample_size = 100
cache_name = f"m4inpatients{sample_size or ''}"
cache_to_disk = f"cached_inteface/{cache_name}"
# use_cached =  'cached_inteface/m4inpatients3000' #cache_to_disk        # False # 
use_cached =  False # 'cached_inteface/m4inpatients3000' #cache_to_disk        # False # 


In [5]:
cache_to_disk

'cached_inteface/m4inpatients100'

##### Possible Interface Scheme Configurations

In [6]:
import json
source_scheme = load_dataset_scheme('M4ICU')
interface_schem_options = source_scheme.supported_target_scheme_options
print(json.dumps(interface_schem_options, sort_keys=True, indent=4))



{
    "dx": [
        "DxICD10",
        "DxCCS",
        "DxFlatCCS",
        "DxICD9"
    ],
    "ethnicity": [
        "MIMIC4Eth32",
        "MIMIC4Eth5"
    ],
    "gender": [
        "Gender"
    ],
    "int_input": [
        "MIMICInput",
        "MIMICInputGroups"
    ],
    "int_proc": [
        "MIMICProcedures",
        "MIMICProcedureGroups"
    ],
    "obs": [
        "MIMICObservables"
    ],
    "outcome": [
        "dx_icd9_filter_v1",
        "dx_icd9_filter_v2_groups",
        "dx_icd9_filter_v3_groups",
        "dx_flatccs_mlhc_groups",
        "dx_flatccs_filter_v1"
    ]
}


#### Leading Observable for Early Prediction Task

In [7]:
scheme_df = source_scheme.obs.as_dataframe()
display(scheme_df[scheme_df.desc.str.contains('aki')])

,code,desc
42,o42,aki_stage_smoothed


In [8]:

# Demographic vector attributes
demographic_vector_conf = DemographicVectorConfig(
    age=False, 
    gender=False, 
    ethnicity=False
)

# Leading 
leading_AKI = LeadingObservableConfig(leading_hours=(6, 12, 24, 48, 72),
                                      window_aggregate='max',
                                      scheme=source_scheme.obs,
                                      index=42)

In [9]:
interface_scheme_kw = dict(dx='DxCCS',
                           outcome='dx_flatccs_filter_v1',
                           ethnicity='MIMIC4Eth5')

In [10]:
if use_cached:
    m4inpatients = Patients.load(use_cached)
    splits = m4inpatients.dataset.random_splits([0.8, 0.9], random_seed=42, balanced='admissions')

else:
    with U.modified_environ(DATA_DIR=DATA_DIR), dask.config.set(scheduler='processes', num_workers=12):
        
        # Load dataset
        m4icu_dataset = load_dataset('M4ICU', sample=sample_size)
        # Use training-split for fitting the outlier_remover and the scalers.
        splits = m4icu_dataset.random_splits([0.8, 0.9], random_seed=42, balanced='admissions')
        # Outlier removal
        outlier_remover = m4icu_dataset.fit_outlier_remover(splits[0])
        m4icu_dataset = m4icu_dataset.remove_outliers(outlier_remover)
        
        # Scale
        scalers = m4icu_dataset.fit_scalers(splits[0])
        m4icu_dataset = m4icu_dataset.apply_scalers(scalers)
        

        # Load interface
        m4inpatients = Patients(
            m4icu_dataset, demographic_vector_conf,
            leading_observable_config=leading_AKI,
            **interface_scheme_kw
        ).load_subjects(num_workers=12)

        # Cache to disk
        m4inpatients.save(cache_to_disk, overwrite=True)

/home/asem/GP/ICENODE/notebooks/mimic_icu/../../lib/ehr/dataset.py:1231: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  dob = anchor_date + anchor_age
INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': NOT_FOUND: Could not find registered platform with name: "rocm". Available platform names are: Interpreter CUDA
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': module 'jaxlib.xla_extension' has no attribute 'get_tpu_client'
/home/asem/GP/ICENODE/notebooks/mimic_icu/../../lib/ehr/concepts.py:44: RuntimeWarning: All-NaN slice encountered
  return np.nanmax(x, axis=axis)


In [11]:
len(m4inpatients.subjects)

100

In [12]:
leading_AKI

LeadingObservableConfig(
  index=42,
  leading_hours=(6, 12, 24, 48, 72),
  window_aggregate=<function nan_agg_max>,
  _index2code={
    0:
    'aki_stage_smoothed_next_6hrs',
    1:
    'aki_stage_smoothed_next_12hrs',
    2:
    'aki_stage_smoothed_next_24hrs',
    3:
    'aki_stage_smoothed_next_48hrs',
    4:
    'aki_stage_smoothed_next_72hrs'
  },
  _code2index={
    'aki_stage_smoothed_next_6hrs':
    0,
    'aki_stage_smoothed_next_12hrs':
    1,
    'aki_stage_smoothed_next_24hrs':
    2,
    'aki_stage_smoothed_next_48hrs':
    3,
    'aki_stage_smoothed_next_72hrs':
    4
  }
)

In [13]:
m4inpatients.leading_observable_config.index

42

In [26]:
sub_ids = sorted(m4inpatients.subjects)
sid = 0
aid = 4
adm = m4inpatients.subjects[sub_ids[sid]].admissions[aid]
adm

Admission(
  admission_id=22009484,
  admission_dates=(
    Timestamp('2200-09-30 14:00:00'),
    Timestamp('2200-10-09 14:05:00')
  ),
  dx_codes=CodesVector(
    vec=bool[728](numpy),
    scheme=<lib.ehr.coding_scheme.DxCCS object at 0x7f2e9dc00730>
  ),
  dx_codes_history=CodesVector(
    vec=bool[728](numpy),
    scheme=<lib.ehr.coding_scheme.DxCCS object at 0x7f2e9dc00730>
  ),
  outcome=CodesVector(
    vec=bool[241](numpy),
    scheme=<lib.ehr.coding_scheme.OutcomeExtractor object at 0x7f2e5a102070>
  ),
  observables=[
    InpatientObservables(
      time=f64[3](numpy),
      value=f16[3,60](numpy),
      mask=bool[3,60](numpy)
    ),
    InpatientObservables(
      time=f64[0](numpy),
      value=f16[0,60](numpy),
      mask=bool[0,60](numpy)
    ),
    InpatientObservables(
      time=f64[0](numpy),
      value=f16[0,60](numpy),
      mask=bool[0,60](numpy)
    ),
    InpatientObservables(
      time=f64[0](numpy),
      value=f16[0,60](numpy),
      mask=bool[0,60](numpy)
  

In [27]:
adm.interventions.time

array([  0.      ,  21.75    ,  21.766666,  21.816668,  21.833334,
        21.883333,  22.      ,  22.016666,  22.6     ,  22.65    ,
        23.      ,  23.5     ,  24.516666,  24.566668,  24.583334,
        25.      ,  25.016666,  25.083334,  25.25    ,  25.316668,
        25.333334,  25.35    ,  25.4     ,  25.483334,  25.533333,
        25.566668,  25.75    ,  26.      ,  26.016666,  26.05    ,
        26.066668,  26.266666,  26.516666,  26.75    ,  26.8     ,
        27.266666,  27.75    ,  27.833334,  27.85    ,  28.466667,
        29.016666,  29.766666,  30.75    ,  30.766666,  31.216667,
        31.316668,  31.35    ,  32.166668,  32.216667,  32.25    ,
        32.583332,  32.75    ,  32.866665,  33.783333,  34.      ,
        34.016666,  34.1     ,  34.116665,  34.183334,  34.2     ,
        34.25    ,  34.783333,  34.866665,  34.966667,  35.      ,
        35.3     ,  35.583332,  35.616665,  35.633335,  35.7     ,
        35.716667,  35.733334,  35.75    ,  36.133335,  36.3  

In [32]:
all_obs = adm.observables[0].concat(adm.observables)
all_obs.time

array([ 16.,  18.,  21.,  22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,
        30.,  31.,  32.,  33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,
        41.,  42.,  43.,  44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,
        52.,  53.,  54.,  55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,
        63.,  64.,  65.,  66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,
        74.,  75.,  76.,  77.,  78.,  79.,  80.,  81.,  82.,  83.,  84.,
        85.,  86.,  87.,  88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,
        96.,  97.,  98.,  99., 100., 101., 102., 103., 104., 105., 106.,
       107., 108., 109., 110., 111., 112., 113., 114., 115., 116., 117.,
       118., 119., 120., 121., 122., 123., 124., 126., 137., 161., 185.])

In [31]:
all_obs.value[:, 42]
# value[:, 42]

array([0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
       0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
       0.    , 0.    , 0.    , 0.    , 0.1666, 0.3333, 0.3333, 0.3333,
       0.3333, 0.3333, 0.3333, 0.    , 0.    , 0.    , 0.3333, 0.3333,
       0.3333, 0.6665, 0.    , 0.6665, 0.    , 0.6665, 0.    , 0.6665,
       0.    , 0.6665, 0.    , 0.6665, 0.6665, 0.6665, 0.    , 0.6665,
       0.    , 0.6665, 0.    , 0.6665, 0.    , 0.3333, 0.    , 0.3333,
       0.    , 0.6665, 0.    , 0.6665, 0.    , 0.6665, 0.6665, 0.6665,
       0.    , 0.    , 0.6665, 0.    , 0.6665, 0.6665, 0.    , 0.6665,
       0.6665, 0.    , 0.    , 0.6665, 0.    , 0.    , 0.    , 0.    ,
       0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.3333,
       0.    , 0.    , 0.3333, 0.    , 0.    , 0.1666, 0.    , 0.    ,
       0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    , 0.    ,
       0.    , 0.    , 0.    , 0.    , 0.    , 0.    ], dtype=float16)

In [33]:
all_leading = adm.leading_observable[0].concat(adm.leading_observable)
all_leading.value

array([[0.        , 0.        , 0.16662598, 0.66650391, 0.66650391],
       [0.        , 0.        , 0.33325195, 0.66650391, 0.66650391],
       [0.        , 0.        , 0.33325195, 0.66650391, 0.66650391],
       [0.        , 0.        , 0.33325195, 0.66650391, 0.66650391],
       [0.        , 0.        , 0.33325195, 0.66650391, 0.66650391],
       [0.        , 0.        , 0.33325195, 0.66650391, 0.66650391],
       [0.        , 0.        , 0.33325195, 0.66650391, 0.66650391],
       [0.        , 0.        , 0.33325195, 0.66650391, 0.66650391],
       [0.        , 0.        , 0.33325195, 0.66650391, 0.66650391],
       [0.        , 0.16662598, 0.33325195, 0.66650391, 0.66650391],
       [0.        , 0.33325195, 0.66650391, 0.66650391, 0.66650391],
       [0.        , 0.33325195, 0.66650391, 0.66650391, 0.66650391],
       [0.        , 0.33325195, 0.66650391, 0.66650391, 0.66650391],
       [0.        , 0.33325195, 0.66650391, 0.66650391, 0.66650391],
       [0.        , 0.33325195, 0.

In [36]:
all_leading.mask

array([[ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  Tr

In [16]:
adm.leading_observable[0].mask

array([[ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True]])

In [9]:
# m4inpatients.size_in_bytes() / 1024 ** 3

In [10]:
# val_batch = m4inpatients.device_batch(splits[1])

In [11]:
# tst_batch = m4inpatients.device_batch(splits[2])

In [12]:
# val_batch.size_in_bytes() / 1024 ** 3, tst_batch.size_in_bytes() / 1024 ** 3

In [13]:
# batch = m4inpatients.device_batch(splits[0][:32])

In [14]:
# batch.size_in_bytes() / 1024 ** 3

In [15]:
# len(batch.subjects)

In [16]:
# batch.n_admissions()

In [17]:
# batch.n_segments()

In [18]:
# batch.n_obs_times()

In [19]:
# import numpy as np
# import matplotlib.pyplot as plt

# a = m4inpatients_jax.obs_coocurrence_matrix
# a = np.array(a)
# plt.imshow(a, cmap='hot', interpolation='nearest')
# plt.show()

In [20]:
# s = batch.subjects[splits[0][6]].admissions[0]
# s.observables[0].value

In [21]:
# batch.interval_hours(splits[0][:10])

### التدريب على نموذج المعادلات التفاضلية الاعتيادية العصبية


In [22]:
from lib.ml import (InICENODE, InICENODEDimensions, InpatientEmbeddingDimensions, 
                    InTrainer, TrainerReporting, OptimizerConfig, WarmupConfig)
from lib.metric import  (CodeAUC, UntilFirstCodeAUC, AdmissionAUC,
                      CodeGroupTopAlarmAccuracy, LossMetric, ObsCodeLevelLossMetric)

import jax.random as jrandom

In [23]:
emb_dims = InpatientEmbeddingDimensions(dx=30, inp=15, proc=15, 
                                        demo=0, 
                                        inp_proc_demo=10)
dims = InICENODEDimensions(mem=15, obs=25, emb=emb_dims)
key = jrandom.PRNGKey(0)

m = InICENODE(dims=dims, 
              schemes=m4inpatients.schemes,
              demographic_vector_config=m4inpatients.demographic_vector_config,
              key=key)

INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': NOT_FOUND: Could not find registered platform with name: "rocm". Available platform names are: Interpreter CUDA
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': module 'jaxlib.xla_extension' has no attribute 'get_tpu_client'


In [24]:
# res = m.batch_predict(m4inpatients.device_batch(), leave_pbar=True)

In [25]:
splits = m4inpatients.random_splits([0.9, 0.95], 
                                    balanced='admissions')

trainer = InTrainer(optimizer_config=OptimizerConfig(opt='adam', lr=1e-3),
                    reg_hyperparams=None,
                    epochs=80,
                    batch_size=128,
                    dx_loss='allpairs_sigmoid_rank',
                    obs_loss='mse')

warmup = WarmupConfig(epochs=0.1, 
                      batch_size=8,
                      opt='adam', lr=1e-3, 
                      decay_rate=0.5)

loss_metric =  LossMetric(m4inpatients, 
                          dx_loss=('softmax_bce', 'balanced_focal_softmax_bce', 
                                   'balanced_focal_bce', 'allpairs_exp_rank', 'allpairs_hard_rank', 
                                   'allpairs_sigmoid_rank'),
                         obs_loss=('mse', 'mae', 'rms'))

metrics = [CodeAUC(m4inpatients), 
           AdmissionAUC(m4inpatients), 
           CodeGroupTopAlarmAccuracy(m4inpatients, n_partitions=5, 
                                     top_k_list=[3, 5, 10, 15, 20],
                                     train_split=splits[0]), 
           loss_metric]


reporting = TrainerReporting(output_dir='inicenode',
                             metrics=metrics,
                             console=True,
                             parameter_snapshots=True,
                             config_json=True)

In [ ]:
res = trainer(m, m4inpatients, 
              splits=splits,
              reporting=reporting,
              n_evals=100,
              warmup_config=warmup,
              continue_training=False)

INFO:root:Warming up...


Loading to device: 0subject [00:00, ?subject/s]

  0%|          | 0/1 [00:00<?, ?Epoch/s]

  0%|          | 0/295 [00:00<?, ?Batch/s]

Loading to device:   0%|          | 0/4 [00:00<?, ?subject/s]

...

Embedding:   0%|          | 0/40 [00:00<?, ?subject/s]

  0%|          | 0.00/782.73 [00:00<?, ?longitudinal-days/s]

Loading to device:   0%|          | 0/38 [00:00<?, ?subject/s]

Embedding:   0%|          | 0/38 [00:00<?, ?subject/s]

  0%|          | 0.00/654.30 [00:00<?, ?longitudinal-days/s]

In [ ]:
import numpy as np
import pandas as pd
sigmoid = lambda x: 1 / (1 + np.exp(x))

p_delta = np.array(np.linspace(0, 1, 11))
scales = np.array([2**i for i in range(5)])
p_delta_scaled = np.outer(p_delta, scales)
leading_loss = sigmoid(p_delta_scaled)
lagging_loss = sigmoid(-p_delta_scaled)
df1 = pd.DataFrame(leading_loss, columns=[f'scale:{s}' for s in scales],
                  index=p_delta).rename_axis('p_delta')
df2 = pd.DataFrame(lagging_loss, columns=[f'scale:{s}' for s in scales],
                  index=p_delta).rename_axis('p_delta')
df3 = df1 / df2
df1

In [ ]:
df2

In [ ]:
df3